# RNN text generation

In this notebook, we'll have a look at simple character-level text generation with recurrent neural networks in Keras.

This is largely a simplified version of the Tensorflow tutorial [Text generation with an RNN](https://www.tensorflow.org/tutorials/text/text_generation), which in turn follows in part Andrej Karpathy's
 [The Unreasonable Effectiveness of Recurrent Neural Networks](http://karpathy.github.io/2015/05/21/rnn-effectiveness/).

## Load Shakespeare dataset

We'll use a dataset of texts from Shakespeare provided by Google. The approach would work with any reasonably-sized plain text file.

In [1]:
# Give -nc (--no-clobber) argument so that the file isn't downloaded multiple times 
!wget -nc https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt

--2020-04-02 09:48:32--  https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt
Resolving storage.googleapis.com (storage.googleapis.com)... 108.177.112.128, 2607:f8b0:4001:c12::80
Connecting to storage.googleapis.com (storage.googleapis.com)|108.177.112.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘shakespeare.txt’

shakespeare.txt     100%[===================>]   1.06M  --.-KB/s    in 0.01s   

2020-04-02 09:48:32 (80.0 MB/s) - ‘shakespeare.txt’ saved [1115394/1115394]



This is a simple plain text file, so we'll just read it in directly.

In [2]:
with open('shakespeare.txt') as f:
    text = f.read()

print(len(text))

1115394


Note that the text includes punctuation and newline characters.

In [3]:
print(repr(text[:200]))
print(text[:200])

'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you'
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you


## Vectorize text

First, find the set of unique characters in the text:

In [4]:
characters = sorted(set(text))

# What did we get?
print(len(characters))
print(characters)

65
['\n', ' ', '!', '$', '&', "'", ',', '-', '.', '3', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


Create a mapping from characters to integers and the inverse mapping from those integers back to the characters.

In [0]:
char_to_int = { c: i for i, c in enumerate(characters) }
# no padding needed so 0 is ok
int_to_char = { i: c for c, i in char_to_int.items() }

Let's have a look at that mapping.

In [6]:
from pprint import pprint    # pretty-printer


def truncate_dict(d, count=10):
    # Returns at most count items from the given dictionary.  
    return dict(i for i, _ in zip(d.items(), range(count)))


pprint(truncate_dict(char_to_int, 20))

{'\n': 0,
 ' ': 1,
 '!': 2,
 '$': 3,
 '&': 4,
 "'": 5,
 ',': 6,
 '-': 7,
 '.': 8,
 '3': 9,
 ':': 10,
 ';': 11,
 '?': 12,
 'A': 13,
 'B': 14,
 'C': 15,
 'D': 16,
 'E': 17,
 'F': 18,
 'G': 19}


Map text characters to the (arbitrary) indices defined above:

In [7]:
data = [char_to_int[c] for c in text] # kirjaimet numeroiksi

print(data[:100])

[18, 47, 56, 57, 58, 1, 15, 47, 58, 47, 64, 43, 52, 10, 0, 14, 43, 44, 53, 56, 43, 1, 61, 43, 1, 54, 56, 53, 41, 43, 43, 42, 1, 39, 52, 63, 1, 44, 59, 56, 58, 46, 43, 56, 6, 1, 46, 43, 39, 56, 1, 51, 43, 1, 57, 54, 43, 39, 49, 8, 0, 0, 13, 50, 50, 10, 0, 31, 54, 43, 39, 49, 6, 1, 57, 54, 43, 39, 49, 8, 0, 0, 18, 47, 56, 57, 58, 1, 15, 47, 58, 47, 64, 43, 52, 10, 0, 37, 53, 59]


Next, let's split that data into fixed-length parts for training.

(The +1 is here because we'll be shifting the outputs by one character.)

In [8]:
# many input to many output
sequence_length = 100

sequences = []
for i in range(0, len(data), sequence_length+1):
    sequences.append(data[i:i+sequence_length+1])
sequences.pop()    # drop (likely) incomplete last sequence

# jokainen tulosteen rivi 100 merkkiä (sequence_length) 
for i in range(5):
    print(repr(''.join(int_to_char[c] for c in sequences[i])))

'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'
"now Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us ki"
"ll him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be d"
'one: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citi'


Split into paired inputs and outputs so that each output matches the text of its input, shifted to the right by one character.

In [9]:
inputs, outputs = [], []
for sequence in sequences:
    inputs.append(sequence[:-1])
    outputs.append(sequence[1:])
    
    
print(repr(''.join(int_to_char[c] for c in inputs[0]))) # input 
# jokaisen lisämerkin perusteella yritetään ennustaa seuraava F: ennusta i, Fi: ennusta r, Fir: ennusta s
print(repr(''.join(int_to_char[c] for c in outputs[0])))

'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'
'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '


## Build model

We build a straightforward RNN model where each RNN state outputs unnormalized predictions ("logits"):

* input: sequence of `sequence_length` integers corresponding to characters
* embedding: randomly initialized mapping to `embedding_dim`-dimensional vectors
* rnn: recurrent neural network with `rnn_units`-dimensional state
* output: `num_classes`-dimensional fully connected layer with softmax activation


In [10]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, GRU, Dense


vocab_size = len(characters)
embedding_dim = 256 #values from google example
rnn_units = 1024


input_ = Input(shape=(sequence_length,))
embedding = Embedding(vocab_size, embedding_dim)(input_)
# Note: return_sequences=True
rnn = GRU(rnn_units, return_sequences=True)(embedding) # gated recurrent unit
# Note: no activation function (e.g. softmax)
# return_sequences=True THIS is the difference between one to one from many to many
# Jokainen 100 RNN-yksikkö tuottaa oman outputin (hidden staten lisäksi)
output = Dense(vocab_size)(rnn)
model = Model(inputs=[input_], outputs=[output])

print(model.summary())

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 100)]             0         
_________________________________________________________________
embedding (Embedding)        (None, 100, 256)          16640     
_________________________________________________________________
gru (GRU)                    (None, 100, 1024)         3938304   
_________________________________________________________________
dense (Dense)                (None, 100, 65)           66625     
Total params: 4,021,569
Trainable params: 4,021,569
Non-trainable params: 0
_________________________________________________________________
None


In [11]:
print(sequence_length)
print(vocab_size)
print(vocab_size*embedding_dim) # size of the input matrix

# logits = log probabilities, raw outputs

100
65
16640


Define an appropriate loss function and compile the model.

In [0]:
from tensorflow.keras.losses import sparse_categorical_crossentropy

# Since we are not using softmax we have raw output and need to tune 
# loss function:

# This is just 'sparse_categorical_crossentropy' where from_logits=True
# specifies that the values are unnormalized (no softmax)
def sparse_categorical_crossentropy_with_logits(labels, logits):
    return sparse_categorical_crossentropy(labels, logits, from_logits=True)


model.compile(
    optimizer='adam',
    loss=sparse_categorical_crossentropy_with_logits,
)

Cast inputs and outputs as numpy arrays

In [13]:
import numpy as np

X, Y = np.array(inputs), np.array(outputs)
print(len(inputs), len(inputs[0]))
print(X.shape)

11043 100
(11043, 100)


Train the model. (This will take a while unless running on a GPU.)

In [14]:
epochs = 10
batch_size = 16


history = model.fit(
    X,
    Y,
    epochs=epochs,
    batch_size=batch_size
)

Epoch 1/10
691/691 [==============================] - 57s 83ms/step - loss: 2.0904
Epoch 2/10
691/691 [==============================] - 56s 81ms/step - loss: 1.5259
Epoch 3/10
691/691 [==============================] - 56s 81ms/step - loss: 1.3863
Epoch 4/10
691/691 [==============================] - 56s 82ms/step - loss: 1.3078
Epoch 5/10
691/691 [==============================] - 57s 82ms/step - loss: 1.2444
Epoch 6/10
691/691 [==============================] - 56s 81ms/step - loss: 1.1853
Epoch 7/10
691/691 [==============================] - 56s 81ms/step - loss: 1.1260
Epoch 8/10
691/691 [==============================] - 57s 82ms/step - loss: 1.0663
Epoch 9/10
691/691 [==============================] - 57s 82ms/step - loss: 1.0073
Epoch 10/10
691/691 [==============================] - 56s 81ms/step - loss: 0.9519


## Generate text with trained model

Note that as we set a fixed `sequence_length` to keep things simple, we won't actually be using the RNN state here; we're instead simply always giving the RNN the catenation of the initial seed text and previously generated characters  as input.

In [17]:
import tensorflow as tf


def predict_one_character(input_, temperature=1.0):
    input_ = ' ' * (sequence_length-len(input_)) + input_   # pad to sequence_length
    input_ = input_[-sequence_length:]    # truncate if longer
    X = np.array([[char_to_int[c] for c in input_]])    # note batch dim
    model.reset_states()    # forget state
    Y = model(X)
    Y = tf.squeeze(Y, 0)    # drop batch dim
    Y = Y / temperature   # log probabilities: dividing with constant flattens the distribution
    # if temp = 1.0, model predicts the most probable outputs
    # if temp = high, for exampe 3.0, model predicts jibberish
    # Sample from categorical distribution 
    pred_id = tf.random.categorical(Y, num_samples=1)[-1,0].numpy()
    return int_to_char[pred_id]


seed = 'ROMEO'
string = seed
generated = []
for i in range(500):
    # Lower temperature gives more likely predictions, higher more surprising
    char = predict_one_character(string, temperature=1.0) # change param value here to see what happens
    generated.append(char)
    string += char


print(seed + ''.join(generated))

ROMEO:
Good morrow with him and all.

PETRUCHIO:
Rather the fatal sweeter.

ROMEO:
I am very alone; sure, I speak; my noble father,
To execute this fair amight to Barblone.
To the desire and the house of judging sith,
When like presence of changing joints forth
Against their husbands; but noble looks on his
death; but he hath sulper'd the Titch and Marcius.
The precious Warwick, take her formort.

KING LEWIS XI:
To me, what news, who would he's myself?

First Servingman:
I would myself besides my min
